A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
# Import functions and classes from Scikit-Learn for machine learning tasks
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Lasso

In [2]:
# Get the current working directory (cwd)
cwd = os.getcwd()

# Define the relative path to the CSV file you want to read
file = "datasets\\rental_info.csv"

# Create a full file path by joining the current working directory and the relative file path
path = os.path.join(cwd, file)

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(path)

# Display the first few rows of the DataFrame to inspect the data
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [3]:
# Convert 'rental_date' column to datetime objects
df["rental_date"] = pd.to_datetime(df["rental_date"])

# Convert 'return_date' column to datetime objects
df["return_date"] = pd.to_datetime(df["return_date"])

# Calculate the rental length as the difference between 'return_date' and 'rental_date'
df["rental_length"] = df["return_date"] - df["rental_date"]

# Extract the rental length in days and store it in a new column 'rental_length_days'
df["rental_length_days"] = df['rental_length'].dt.days

# Display the first few rows of the DataFrame to inspect the changes
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length,rental_length_days
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3 days 20:46:00,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 20:05:00,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7 days 05:44:00,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 02:24:00,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4 days 01:05:00,4


In [4]:
# Create a new column 'deleted_scenes' with binary values (1 or 0)
# If "Deleted Scenes" is found in 'special_features', set the value to 1, otherwise set it to 0
df["deleted_scenes"] = np.where(df["special_features"].str.contains("Deleted Scenes"), 1, 0)

# Create a new column 'behind_the_scenes' with binary values (1 or 0)
# If "Behind the Scenes" is found in 'special_features', set the value to 1, otherwise set it to 0
df["behind_the_scenes"] = np.where(df["special_features"].str.contains("Behind the Scenes"), 1, 0)

In [5]:
# List of column names to be dropped from the DataFrame
to_drop = ["rental_date", "return_date", "special_features", "rental_length", "rental_length_days"]

# Create a new DataFrame 'X' by dropping the specified columns along the specified axis (columns axis, axis=1)
X = df.drop(to_drop, axis=1)

# Create a Series 'y' containing the values from the "rental_length_days" column
y = df["rental_length_days"]

# Print the shape (dimensions) of the 'X' and 'y' data
print(X.shape, y.shape)

(15861, 14) (15861,)


In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_preds = model.predict(X_test)

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, y_preds)
print("Mean Squared Error:", mse)

Mean Squared Error: 2.941723864697592


In [7]:
# Define the model and its hyperparameter search space
lasso_model = Lasso()

hyperparameters = {
    "alpha": np.arange(1, 1001, 1, dtype=float),
    "max_iter": np.arange(1, 1001, 1)
}

# Create a randomized search cross-validation object
randomized_search = RandomizedSearchCV(
    estimator=lasso_model,
    param_distributions=hyperparameters,
    cv=5,  # 5-fold cross-validation
    random_state=9,
    n_iter=100  # Number of parameter combinations to try
)

# Fit the randomized search to the training data
randomized_search.fit(X_train, y_train)

# Get the best hyperparameters from the search
best_params = randomized_search.best_params_

# Create a new Lasso model with the best hyperparameters
lasso_model = Lasso(alpha=best_params["alpha"], max_iter=best_params["max_iter"])

# Fit the Lasso model to the training data
lasso_model.fit(X_train, y_train)

# Make predictions on the test data
y_preds = lasso_model.predict(X_test)

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, y_preds)
print("Mean Squared Error:", mse)

Mean Squared Error: 5.151156811267764


In [8]:
# Random Forest Regressor
param = {
    "n_estimators": np.arange(1,101,1),
    "max_depth": np.arange(1,11,1)
}

rf_model = RandomForestRegressor()

cv = RandomizedSearchCV(rf_model, param_distributions=param, cv=5, random_state=9)

cv.fit(X_train, y_train)

rf_model = RandomForestRegressor(n_estimators=cv.best_params_["n_estimators"], max_depth=cv.best_params_["max_depth"])
rf_model.fit(X_train, y_train)
y_preds = rf_model.predict(X_test)
# Calculate and print the mean squared error
mse = mean_squared_error(y_test, y_preds)
print("Mean Squared Error:", mse)

Mean Squared Error: 2.222254283175254


In [9]:
# Define a dictionary of hyperparameters to search over
parameters = {
    "min_samples_leaf": np.arange(1, 11, 1),
    "max_leaf_nodes": np.arange(1, 11, 1),
    "min_samples_split": np.arange(1, 11, 1)
}

# Create an instance of the Decision Tree Regressor model
d_model = DecisionTreeRegressor()

# Create an instance of RandomizedSearchCV for hyperparameter tuning
# It will search over the specified hyperparameter space using cross-validation
cv = RandomizedSearchCV(d_model, param_distributions=parameters, cv=5, random_state=9)

# Fit the RandomizedSearchCV object to the training data to find the best hyperparameters
cv.fit(X_train, y_train)

# Get the best hyperparameters found during the randomized search
p = cv.best_params_

# Create a new Decision Tree Regressor model using the best hyperparameters found
d_model = DecisionTreeRegressor(
    min_samples_leaf=p["min_samples_leaf"],
    max_leaf_nodes=p["max_leaf_nodes"],
    min_samples_split=p["min_samples_split"]
)

# Fit the new model to the training data
d_model.fit(X_train, y_train)

# Make predictions on the test data using the tuned Decision Tree model
ypreds = d_model.predict(X_test)

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, y_preds)
print("Mean Squared Error:", mse)

Mean Squared Error: 2.222254283175254


c:\Users\alexc\miniconda3\envs\minimal_ds\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\alexc\miniconda3\envs\minimal_ds\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\alexc\miniconda3\envs\minimal_ds\lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "c:\Users\alexc\miniconda3\envs\minimal_ds\lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_paramet

In [10]:
best_model = d_model
best_mse = mse